In [1]:
import os
import cv2 as cv
import networkx as nx
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
import numpy as np

In [2]:
#This function loads images from a specific folder
def load_images_from_folder(folder):
    images = []
    for filename in os.listdir(folder):
        img = cv.imread(os.path.join(folder,filename))
        img = cv.cvtColor(img, cv.COLOR_BGR2RGB)
        if img is not None:
            images.append(img)
    return images

In [3]:
def build_and_print_graph(imgs, adj_matrix, weight_matrix, save_images = True, graph_dir = "output", graph_name = "graph"):
    G=nx.Graph()
    n=len(imgs)
    for i in range(0,n):
        G.add_node(i,image = imgs[i])
    for i in range(n): 
         for j in range(n): 
            if adj_matrix[i,j] == 1:
                G.add_edge(i,j, weight=weight_matrix[i,j]) 
    pos=nx.circular_layout(G)
    fig=plt.figure(figsize=(15,15))
    ax=plt.subplot(111)
    ax.set_aspect('equal')
    nx.draw(G,pos,ax=ax, width = 3, node_size=900,with_labels = True, 
            edgecolors='red', node_color='lightgray', connectionstyle='arc3, rad = 0.1')
    
    labels = nx.get_edge_attributes(G,'weight')
    nx.draw_networkx_edge_labels(G,pos,edge_labels=labels)
    plt.xlim(-1.5,1.5)
    plt.ylim(-1.5,1.5)
    trans=ax.transData.transform
    trans2=fig.transFigure.inverted().transform
    piesize=0.08 # this is the image size
    p2=piesize/0.9 #this is the image center
    for g in G:
        xx,yy=trans(pos[g]) # figure coordinates
        xa,ya=trans2((xx,yy)) # axes coordinates
        a = plt.axes([xa-p2,ya-p2, piesize, piesize])
        a.set_aspect('equal')
        a.imshow(G.nodes[g]['image'])
        a.axis('off')
    ax.axis('off')
    if save_images:
        plt.savefig(os.path.join(graph_dir,f'{graph_name}.png'))
    return G

In [4]:
def build_and_print_multi_graph(adj_matrix, save_images = True, graph_dir = "output", graph_name = "graph"):
    graph = nx.from_numpy_matrix(adj_matrix, parallel_edges=True, create_using=nx.MultiDiGraph)
    pos=nx.circular_layout(graph)
    nx.draw(graph, pos=pos, with_labels=True, connectionstyle='arc3, rad = 0.1')
    if save_images:
        plt.savefig(os.path.join(graph_dir,f'{graph_name}.png'))
    return graph

In [5]:
def print_graph(graph, save_images = True, graph_dir = "output", graph_name = "graph"):
    pos=nx.kamada_kawai_layout(graph)
    fig=plt.figure(figsize=(10,10))
    ax=plt.subplot(111)
    ax.set_aspect('equal')
    nx.draw(graph,pos,ax=ax, width = 3, node_size=900,with_labels = True, edgecolors='red', node_color='lightgray')
    if save_images:
        plt.savefig(os.path.join(graph_dir,f'{graph_name}.png'))

In [6]:
def get_reference_node(adj_matrix):
    return np.argmax(np.sum(adj_matrix, axis=0))

In [7]:
#This function splits a 3nx3n matrix into a set of 3x3 matrices
def split_states(x):
    x_small = x.transpose()
    res = [ x_small[:,i*3:(i+1)*3].transpose() for i in range(x_small.shape[1]//3)]
    return res

In [8]:
def get_normalization_matrix(imgs):
    n = len(imgs) #Number of images
    RESCALE = 1./np.max(imgs[0].shape)
    T_norm = np.diag([RESCALE,RESCALE,1])
    return T_norm

In [10]:
class ImageStitcher:
    
    def __init__(self, imgs):
        self.imgs=imgs

    def get_homographies_from_states(self, U, idx_ref=0):
        #Compute the homography of each image w.r.t. the reference one
        H = [np.dot(U[i],np.linalg.inv(U[idx_ref])) for i in range(len(self.imgs))]
        return H
    def __denormalize_homographies(self, H, T_norm):
        H_denorm = [ np.linalg.inv(T_norm) @ h @ T_norm  for h in H]
        return H_denorm
    
    def __compute_translation(self, idxs, H, size =  [5000,10000]):
    
        min_x = 0
        min_y = 0
        max_x = 0
        max_y = 0

        #For each image
        for i in idxs:

             ## translate
            (Height, Width, _) = self.imgs[i].shape

            # Taking the matrix of initial coordinates of the corners of the secondary image
            # Stored in the following format: [[x1, x2, x3, x4], [y1, y2, y3, y4], [1, 1, 1, 1]]
            # Where (xt, yt) is the coordinate of the i th corner of the image. 
            InitialMatrix = np.array([[0, Width - 1, Width - 1, 0],
                                      [0, 0, Height - 1, Height - 1],
                                      [1, 1, 1, 1]])

            # Finding the final coordinates of the corners of the image after transformation.
            # NOTE: Here, the coordinates of the corners of the frame may go out of the 
            # frame(negative values). We will correct this afterwards by updating the 
            # homography matrix accordingly.
            FinalMatrix = np.dot(np.linalg.inv(H[i]), InitialMatrix)

            [x, y, c] = FinalMatrix
            x = np.divide(x, c)
            y = np.divide(y, c)

            c_min_x = min(x)
            c_min_y = min(y)
            c_max_x = max(x)
            c_max_y = max(y)

            if c_min_x < min_x:
                min_x = c_min_x

            if c_min_y < min_y:
                min_y = c_min_y

            if c_max_x > max_x:
                max_x = c_max_x

            if c_max_y > max_y:
                max_y = c_max_y

       
        t = [min_x,min_y]
        dy = int(min(max_y - min_y, size[0]))
        dx = int(min(max_x - min_x, size[1]))
        size = [dy,dx]

        Ht = np.array([[1,0,t[0]],[0,1,t[1]],[0,0,1]]) # translate
        return Ht, size
    
    def stitch_images(self, H, idx_ref, T_norm, idxs=None, beautify=True, size = [5000,10000]):
        
        num = len(self.imgs)
        if not idxs:
            idxs = list(range(num))
        
        H_denorm = self.__denormalize_homographies(H, T_norm)
        
        if beautify:
            Ht, size = self.__compute_translation(idxs, H_denorm, size)
        else:
            Ht = np.eye(3)
        
        H_norm_transl = [(hn @ T_norm @ Ht @  np.linalg.inv(T_norm)) for hn in H]
        stitch = np.zeros(size + [3], dtype=np.uint8)
       
        #For each image
        for i in idxs:

            #Apply the homography        

            img_proj = cv.warpPerspective(self.imgs[i], np.linalg.inv(H_denorm[i] @ Ht) , size[::-1])


            #Use maximum as stitch operator (very simple stitching mechanism)
            stitch = np.maximum(stitch,img_proj)


        return H_norm_transl, stitch
    
    
    def __compute_translation_2(self, idxs, H, size =  [5000,10000]):
    
        min_x = 0
        min_y = 0
        max_x = 0
        max_y = 0

        #For each image
        for i in idxs:

             ## translate
            (Height, Width, _) = self.imgs[i].shape

            # Taking the matrix of initial coordinates of the corners of the secondary image
            # Stored in the following format: [[x1, x2, x3, x4], [y1, y2, y3, y4], [1, 1, 1, 1]]
            # Where (xt, yt) is the coordinate of the i th corner of the image. 
            InitialMatrix = np.array([[0, Width - 1, Width - 1, 0],
                                      [0, 0, Height - 1, Height - 1],
                                      [1, 1, 1, 1]])

            # Finding the final coordinates of the corners of the image after transformation.
            # NOTE: Here, the coordinates of the corners of the frame may go out of the 
            # frame(negative values). We will correct this afterwards by updating the 
            # homography matrix accordingly.
            FinalMatrix = np.dot(H[i], InitialMatrix)

            [x, y, c] = FinalMatrix
            x = np.divide(x, c)
            y = np.divide(y, c)

            c_min_x = min(x)
            c_min_y = min(y)
            c_max_x = max(x)
            c_max_y = max(y)

            if c_min_x < min_x:
                min_x = c_min_x

            if c_min_y < min_y:
                min_y = c_min_y

            if c_max_x > max_x:
                max_x = c_max_x

            if c_max_y > max_y:
                max_y = c_max_y

       
        t = [-min_x,-min_y]
        dy = int(min(max_y - min_y, size[0]))
        dx = int(min(max_x - min_x, size[1]))
        size = [dy,dx]
        
        print(t)

        Ht = np.array([[1,0,t[0]],[0,1,t[1]],[0,0,1]]) # translate
        return Ht, size
    
    def stitch_images_denorm(self, H, idx_ref, idxs=None, beautify=True, size = [5000,10000]):
        
        num = len(self.imgs)
        if not idxs:
            idxs = list(range(num))
        
        if beautify:
            Ht, size = self.__compute_translation_2(idxs, H, size)
        else:
            Ht = np.eye(3)
        
        stitch = np.zeros(size + [3], dtype=np.uint8)
       
        #For each image
        for i in idxs:

            #Apply the homography        

            img_proj = cv.warpPerspective(self.imgs[i], Ht @  H[i] , size[::-1])


            #Use maximum as stitch operator (very simple stitching mechanism)
            stitch = np.maximum(stitch,img_proj)

        return Ht, stitch